In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataloaders.midair import DataLoaderMidAir as MidAir
from dataloaders import DataloaderParameters

param = DataloaderParameters({'_comment': '/Users/pascalleroy/Documents/m4depth/M4Depth/relative paths should be written relative to this file', 'midair': '/Users/pascalleroy/Documents/m4depth/M4Depth/datasets/MidAir', 'kitti-raw': '/Users/pascalleroy/Documents/m4depth/M4Depth/datasets/Kitti', 'tartanair': '/Users/pascalleroy/Documents/m4depth/M4Depth/datasets/TartanAir'},
    'data/midair/small_test_data',
    None,
    4,
    True)

chosen_dataloader = MidAir()
chosen_dataloader.get_dataset("eval", param, batch_size=1)
data = chosen_dataloader.dataset

2022-09-08 15:05:47.516041: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
from importlib import reload
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras as ks
from keras.utils import vis_utils
from metrics import AbsRelError, SqRelError, RootMeanSquaredError, \
    RootMeanSquaredLogError, ThresholdRelError

from callbacks import CustomCheckpointCallback

print(tf.__version__)
from m4depth_network import M4Depth, M4depthAblationParameters

ablation_settings = M4depthAblationParameters()

print(tf.config.list_physical_devices('GPU'))
model = M4Depth(nbre_levels=6, ablation_settings=ablation_settings)

weights_dir = os.path.join("pretrained_weights/midair/","best")
model_checkpoint_cbk = CustomCheckpointCallback(weights_dir, resume_training=True)
model.compile(metrics=[AbsRelError(),
                       SqRelError(),
                       RootMeanSquaredError(),
                       RootMeanSquaredLogError(),
                       ThresholdRelError(1), ThresholdRelError(2), ThresholdRelError(3)])

metrics = model.evaluate(data, callbacks=[model_checkpoint_cbk])
print(metrics)

2.9.1
[]
Checkpoint save directory: pretrained_weights/midair/best
Latest checkpoint found: pretrained_weights/midair/best/cp-0071.ckpt
Restoring weights from pretrained_weights/midair/best/cp-0071.ckpt
Temporal memory instanciation
Temporal memory instanciation
Temporal memory instanciation
Temporal memory instanciation
Temporal memory instanciation
Temporal memory instanciation
type(traj_samples) <class 'list'> 1
--**--prev_f_maps None
Seq sample 0
ok TensorShape([1, 48, 48, 1]) TensorShape([1, 48, 48, 64])
ok TensorShape([1, 12, 12, 1]) TensorShape([1, 12, 12, 128])
ok TensorShape([1, 192, 192, 1]) TensorShape([1, 192, 192, 16])
ok TensorShape([1, 24, 24, 1]) TensorShape([1, 24, 24, 96])
ok TensorShape([1, 96, 96, 1]) TensorShape([1, 96, 96, 32])
ok TensorShape([1, 6, 6, 1]) TensorShape([1, 6, 6, 192])
l, fmaps, 0 - None 
--**-- post init prev_f_maps <tf.Variable 'DepthEstimator_seq/depth_estimator_level_18/prev_f_maps:0' shape=(1, 192, 192, 16) dtype=float32>
--**-- post init prev_

In [8]:
from m4depth_simplified_network import M4Depth as M4DepthSimplified

model = M4DepthSimplified(nbre_levels=6)

weights_dir = os.path.join("pretrained_weights/midair/","best")
model_checkpoint_cbk = CustomCheckpointCallback(weights_dir, resume_training=True)
model.compile(metrics=[AbsRelError(),
                       SqRelError(),
                       RootMeanSquaredError(),
                       RootMeanSquaredLogError(),
                       ThresholdRelError(1), ThresholdRelError(2), ThresholdRelError(3)])

metrics = model.evaluate(data, callbacks=[model_checkpoint_cbk])
print(metrics)

Checkpoint save directory: pretrained_weights/midair/best
Latest checkpoint found: pretrained_weights/midair/best/cp-0071.ckpt
Restoring weights from pretrained_weights/midair/best/cp-0071.ckpt
Temporal memory instanciation
Temporal memory instanciation
Temporal memory instanciation
Temporal memory instanciation
Temporal memory instanciation
Temporal memory instanciation
type(traj_samples) <class 'list'> 1
--**--prev_f_maps None
Seq sample 0
ok TensorShape([1, 6, 6, 1]) TensorShape([1, 6, 6, 192])
ok TensorShape([1, 96, 96, 1]) TensorShape([1, 96, 96, 32])
ok TensorShape([1, 12, 12, 1]) TensorShape([1, 12, 12, 128])
ok TensorShape([1, 192, 192, 1]) TensorShape([1, 192, 192, 16])
ok TensorShape([1, 48, 48, 1]) TensorShape([1, 48, 48, 64])
ok TensorShape([1, 24, 24, 1]) TensorShape([1, 24, 24, 96])
--**-- post init prev_f_maps <tf.Variable 'DepthEstimator_seq/depth_estimator_level_31/prev_f_maps:0' shape=(1, 96, 96, 32) dtype=float32>
--**-- post init prev_f_maps <tf.Variable 'DepthEstim

In [15]:
model.summary()


Model: "m4_depth_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_pyramid_5 (FeatureP  multiple                 1022192   
 yramid)                                                         
                                                                 
 depth_estimator_pyramid_5 (  multiple                 4632018   
 DepthEstimatorPyramid)                                          
                                                                 
Total params: 5,654,211
Trainable params: 4,492,238
Non-trainable params: 1,161,973
_________________________________________________________________
